In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def capture_on_keypress(video_path, key='n'):
    cap = cv.VideoCapture(video_path)
    images = []  # List to store captured images
    print("Press '{}' to capture a frame, 'q' to quit.".format(key))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if no frame is captured
        
        # Convert frame to grayscale
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
        # Display the frame
        cv.imshow('Video', gray_frame)
        
        # Wait for the 'n' key or 'q' to quit
        if cv.waitKey(1) & 0xFF == ord(key):
            images.append(gray_frame)  # Capture the frame on 'n'
            print("Frame captured.")
        elif cv.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv.destroyAllWindows()
    return images

# Usage
video_path = 'your_video_file.mp4'  # Specify your video path here
captured_images = capture_on_keypress(video_path, key='n')

# Display captured images, for verification
for i, img in enumerate(captured_images):
    cv.imshow('Captured Image {}'.format(i+1), img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
MIN_MATCH_COUNT = 10
# img1 = cv.imread('box.png', cv.IMREAD_GRAYSCALE)          # queryImage
# img2 = cv.imread('box_in_scene.png', cv.IMREAD_GRAYSCALE) # trainImage

img1 = captured_images[0]         # queryImage
img2 = captured_images[1]         # trainImage

# Initiate SIFT detector
sift = cv.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

In [ ]:
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None
    
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
plt.imshow(img3, 'gray'),plt.show()